In [ ]:
!pip install ipywidgets==7.7.1

In [ ]:
!pip install datasets

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Получаю датасет ru_go_emotions


In [ ]:
from datasets import load_dataset

ds = load_dataset("SkyWater21/ru_go_emotions_ekman")

In [ ]:
print(ds)

In [ ]:
train = ds['train']
valid = ds['validation']
test = ds['test']

In [ ]:
dct = {
0: "восхищение",
1: "веселье",
2: "злость",
3: "раздражение",
4: "одобрение",
5: "забота",
6: "непонимание",
7: "любопытство",
8: "желание",
9: "разочарование",
10: "неодобрение",
11: "отвращение",
12: "смущение",
13: "возбуждение",
14: "страх",
15: "признательность",
16: 'горе',
17: "радость",
18: "любовь",
19: "нервозность",
20: "оптимизм",
21: "гордость",
22: "осознание",
23: "облегчение",
24: "раскаяние",
25: "грусть",
26: "удивление",
27: "нейтральность"
}

In [ ]:
import pandas as pd

train = pd.DataFrame.from_dict(train)
train = train.loc[:, ['ru_text', 'labels']]

valid = pd.DataFrame.from_dict(valid)
valid = valid.loc[:, ['ru_text', 'labels']]

test = pd.DataFrame.from_dict(test)
test = test.loc[:, ['ru_text', 'labels']]

In [ ]:
print(test)

# Очистка текста


In [ ]:
!pip install nltk

In [ ]:
!pip install pymorphy2

In [ ]:
import re
import nltk.corpus
import pymorphy2
from nltk.corpus import stopwords
import string

nltk.download('stopwords')
stop = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

In [ ]:
def cleaning(text):
    text = text.lower()
    text = "".join([el.strip(string.punctuation+'–—') for el in text if el not in string.punctuation])
    text = " ".join([morph.parse(word)[0].normal_form for word in text.split() if word not in (stop)])
    return text

In [ ]:
print(cleaning(train['ru_text'][0]))

In [ ]:
train['ru_text'] = train['ru_text'].apply(cleaning)

valid['ru_text'] = valid['ru_text'].apply(cleaning)

test['ru_text'] = test['ru_text'].apply(cleaning)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['ru_text'].tolist())
tokenizer.fit_on_texts(valid['ru_text'].tolist())

In [ ]:
sequences = tokenizer.texts_to_sequences(train['ru_text'].tolist())
max_length = max([len(seq) for seq in sequences])
x = pad_sequences(sequences, maxlen=max_length)
sequences1 = tokenizer.texts_to_sequences(valid['ru_text'].tolist())
max_length1 = max([len(seq) for seq in sequences1])
x_v = pad_sequences(sequences1, maxlen=max_length)

# Векторизация через эмбеддинг-слои Keras

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=28, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
hist = model.fit(x, y_train, epochs=10, batch_size=32, validation_data=(x_v, y_val))

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=64, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=64, activation="relu"))
model.add(Dense(units=28, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
hist = model.fit(x, y_train, epochs=10, batch_size=128, validation_data=(x_v, y_val))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 128, return_sequences = True, input_shape = (x.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.25))

regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.25))

regressor.add(LSTM(units = 128))
regressor.add(Dropout(0.25))

regressor.add(Dense(units = 28))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])

reshaped_x = x.reshape((x.shape[0], x.shape[1], 1))
reshaped_x_v = x_v.reshape((x_v.shape[0], x_v.shape[1], 1))

hist1 = regressor.fit(reshaped_x, y_train, epochs = 25, batch_size = 256, validation_data=(reshaped_x_v, y_val))

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 128, return_sequences = True, input_shape = (x.shape[1], 1)))
regressor.add(Dropout(0.25))

regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.25))

regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.25))

regressor.add(LSTM(units = 128))
regressor.add(Dropout(0.25))

regressor.add(Dense(units = 28))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])

reshaped_x = x.reshape((x.shape[0], x.shape[1], 1))
reshaped_x_v = x_v.reshape((x_v.shape[0], x_v.shape[1], 1))

hist1 = regressor.fit(reshaped_x, y_train, epochs = 25, batch_size = 256, validation_data=(reshaped_x_v, y_val))

In [ ]:
history_dict = hist1.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, 'bo', label='Потери при Обучении')
plt.plot(epochs, val_loss_values, 'b', label='Потери при Проверке')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.clf()
acc = history_dict["accuracy"]
val_acc = history_dict["val_accuracy"]
plt.plot(epochs, acc, "bo", label="Точность на этапе обучения")
plt.plot(epochs, val_acc, "b", label="Точность на этапе проверки")
plt.title("Точность")
plt.xlabel("Эпохи")
plt.ylabel("Точность")
plt.legend()
plt.show()

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 128, return_sequences = True, input_shape = (x.shape[1], 1)))
regressor.add(Dropout(0.5))

regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.5))

regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.5))

regressor.add(LSTM(units = 128))
regressor.add(Dropout(0.5))

regressor.add(Dense(units = 28))

regressor.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

reshaped_x = x.reshape((x.shape[0], x.shape[1], 1))
reshaped_x_v = x_v.reshape((x_v.shape[0], x_v.shape[1], 1))

hist1 = regressor.fit(reshaped_x, y_train, epochs = 25, batch_size = 256, validation_data=(reshaped_x_v, y_val))

In [ ]:
model = Sequential()
model.add(layers.Input(shape=(30,)))
model.add(layers.Reshape((-1, 1)))
# Выходом GRU будет 3D тензор размера (batch_size, timesteps, 256)
model.add(layers.GRU(128, return_sequences=True))
model.add(layers.GRU(128, return_sequences=True))
# Выходом SimpleRNN будет 2D тензор размера (batch_size, 128)
model.add(layers.SimpleRNN(128))
model.add(layers.Dense(28))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

history = model.fit(x,
                    y_train,
                    epochs=100,
                    batch_size=256,
                    validation_data=(x_v, y_val))

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 128, return_sequences = True, input_shape = (x.shape[1], 1)))
regressor.add(Dropout(0.5))

regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.5))

regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.5))

regressor.add(LSTM(units = 128))
regressor.add(Dropout(0.5))

regressor.add(Dense(units = 28))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])

reshaped_x = x.reshape((x.shape[0], x.shape[1], 1))
reshaped_x_v = x_v.reshape((x_v.shape[0], x_v.shape[1], 1))

hist1 = regressor.fit(reshaped_x, y_train, epochs = 100, batch_size = 256, validation_data=(reshaped_x_v, y_val))

# Векторизация через Word2Vec

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
from gensim.models import Word2Vec

sentences = [sentence.split() for sentence in train['ru_text']]
w2v_model = Word2Vec(sentences, window=5, min_count=5, workers=4)

In [ ]:
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

x_train = np.array([vectorize(sentence) for sentence in train['ru_text']])
x_val = np.array([vectorize(sentence) for sentence in valid['ru_text']])
x_test = np.array([vectorize(sentence) for sentence in test['ru_text']])

In [ ]:
zero_1 = np.zeros((train['ru_text'].shape[0], len(dct)))
zero_2 = np.zeros((valid['ru_text'].shape[0], len(dct)))

In [ ]:
def to_labels(mass, labl):
    for i in range(mass.shape[0]):
        for j in labl[i]:
            mass[i][j] = 1
    return mass

y_train = to_labels(zero_1, train['labels'])
y_val = to_labels(zero_2, valid['labels'])

# Векторизация через мешок слов

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(train['ru_text'])

# Виды используемых для обучения моделей

## Простая нейронная сеть из Dense слоёв

In [ ]:
pip install -U jax jaxlib

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [ ]:
model = keras.Sequential([
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(28, activation='softmax')]
)

callbacks = [keras.callbacks.ModelCheckpoint("lstm16_1.keras", save_best_only=True)]

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['precision', 'recall', 'accuracy'])

history = model.fit(x_train,
                    y_train,
                    epochs=25,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    callbacks=callbacks)

In [ ]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, 'bo', label='Потери при Обучении')
plt.plot(epochs, val_loss_values, 'b', label='Потери при Проверке')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.clf()
acc = history_dict["accuracy"]
val_acc = history_dict["val_accuracy"]
plt.plot(epochs, acc, "bo", label="Точность на этапе обучения")
plt.plot(epochs, val_acc, "b", label="Точность на этапе проверки")
plt.title("Точность")
plt.xlabel("Эпохи")
plt.ylabel("Точность")
plt.legend()
plt.show()

plt.clf()
acc = history_dict["precision"]
val_acc = history_dict["val_precision"]
plt.plot(epochs, acc, "bo", label="Точность на этапе обучения")
plt.plot(epochs, val_acc, "b", label="Точность на этапе проверки")
plt.title("Точность")
plt.xlabel("Эпохи")
plt.ylabel("Точность")
plt.legend()
plt.show()

plt.clf()
acc = history_dict["recall"]
val_acc = history_dict["val_recall"]
plt.plot(epochs, acc, "bo", label="Точность на этапе обучения")
plt.plot(epochs, val_acc, "b", label="Точность на этапе проверки")
plt.title("Точность")
plt.xlabel("Эпохи")
plt.ylabel("Точность")
plt.legend()
plt.show()

### LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.25))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.25))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.25))

regressor.add(Dense(units = 28))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['precision', 'recall', 'accuracy'])

reshaped_x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
reshaped_x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))

history1 = regressor.fit(reshaped_x_train, y_train, epochs = 25, batch_size = 128, validation_data=(reshaped_x_val, y_val))

In [ ]:
history_dict = history1.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, 'bo', label='Потери при Обучении')
plt.plot(epochs, val_loss_values, 'b', label='Потери при Проверке')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.clf()
acc = history_dict["accuracy"]
val_acc = history_dict["val_accuracy"]
plt.plot(epochs, acc, "bo", label="Точность на этапе обучения")
plt.plot(epochs, val_acc, "b", label="Точность на этапе проверки")
plt.title("Точность")
plt.xlabel("Эпохи")
plt.ylabel("Точность")
plt.legend()
plt.show()

plt.clf()
acc = history_dict["precision"]
val_acc = history_dict["val_precision"]
plt.plot(epochs, acc, "bo", label="Точность на этапе обучения")
plt.plot(epochs, val_acc, "b", label="Точность на этапе проверки")
plt.title("Точность")
plt.xlabel("Эпохи")
plt.ylabel("Точность")
plt.legend()
plt.show()

plt.clf()
acc = history_dict["recall"]
val_acc = history_dict["val_recall"]
plt.plot(epochs, acc, "bo", label="Точность на этапе обучения")
plt.plot(epochs, val_acc, "b", label="Точность на этапе проверки")
plt.title("Точность")
plt.xlabel("Эпохи")
plt.ylabel("Точность")
plt.legend()
plt.show()

## 

## SimpleRNN

In [ ]:
model = Sequential()
model.add(layers.Input(shape=(100,)))
model.add(layers.Reshape((-1, 1)))
# Выходом GRU будет 3D тензор размера (batch_size, timesteps, 256)
model.add(layers.GRU(64, return_sequences=True))
model.add(layers.GRU(128, return_sequences=True))
# Выходом SimpleRNN будет 2D тензор размера (batch_size, 128)
model.add(layers.SimpleRNN(128))
model.add(layers.Dense(28))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['precision', 'recall', 'accuracy'])
model.summary()

callbacks = [keras.callbacks.ModelCheckpoint("flatten64_1.keras", save_best_only=True)]

history = model.fit(x_train,
                    y_train,
                    epochs=50,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    callbacks=callbacks)

In [ ]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, 'bo', label='Потери при Обучении')
plt.plot(epochs, val_loss_values, 'b', label='Потери при Проверке')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
​
plt.clf()
acc = history_dict["accuracy"]
val_acc = history_dict["val_accuracy"]
plt.plot(epochs, acc, "bo", label="Точность на этапе обучения")
plt.plot(epochs, val_acc, "b", label="Точность на этапе проверки")
plt.title("Точность")
plt.xlabel("Эпохи")
plt.ylabel("Точность")
plt.legend()
plt.show()
​
plt.clf()
acc = history_dict["precision"]
val_acc = history_dict["val_precision"]
plt.plot(epochs, acc, "bo", label="Точность на этапе обучения")
plt.plot(epochs, val_acc, "b", label="Точность на этапе проверки")
plt.title("Точность")
plt.xlabel("Эпохи")
plt.ylabel("Точность")
plt.legend()
plt.show()
​
plt.clf()
acc = history_dict["recall"]
val_acc = history_dict["val_recall"]
plt.plot(epochs, acc, "bo", label="Точность на этапе обучения")
plt.plot(epochs, val_acc, "b", label="Точность на этапе проверки")
plt.title("Точность")
plt.xlabel("Эпохи")
plt.ylabel("Точность")
plt.legend()
plt.show()


## Conv1D и MaxPooling

In [ ]:
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(100, embedding_dim, input_length=100))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(28, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['precision', 'recall', 'accuracy'])
model.summary()

history = model.fit(x_train,
                    y_train,
                    epochs=50,
                    batch_size=256,
                    validation_data=(x_val, y_val),
                    callbacks=callbacks)

## BERT

In [ ]:
print(train['ru_text'].shape)

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
import pandas as pd
# Загрузка предварительно обученной модели BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def vectorize(texts):
    # Токенизация текста и преобразование в тензор
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    # Получение эмбеддингов текста с помощью модели BERT
    outputs = model(**inputs)

    # Получение представлений текста (последний скрытый слой)
    last_hidden_states = outputs.last_hidden_state
    return last_hidden_states.detach().numpy()

x_val = valid['ru_text'].copy()

for i in range(len(valid['ru_text'])):
    if i%1000==0:
        print(i)
    x_val[i] = np.array(vectorize(valid['ru_text'][i]))

In [ ]:
print(x_val.values)

In [ ]:
x_train.to_csv(/kaggle/input/data-train)

In [ ]:
import pandas as pd
x_train = pd.read_csv('/kaggle/input/data-train/data_train.csv')

In [ ]:
import numpy as np
x_train = x_train['ru_text'].values

In [ ]:
x_train = x_train['ru_text']

In [ ]:
for i in range(x_train.shape[0]):
    print(x_train[i][0])
    break

In [ ]:
print(x_train.values)

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt

model = tf.keras.Sequential([
    layers.Flatten(input_shape=(x_train.shape[1:])),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(28, activation='softmax')]
)

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['precision', 'recall', 'accuracy'])

history = model.fit(x_train,
                    y_train,
                    epochs=25,
                    batch_size=512,
                    validation_data=(x_val, y_val))

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")

x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(28, activation="sigmoid", name="predictions")(x)

model = keras.Model(inputs, predictions)

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['precision', 'recall', 'accuracy'])

history = model.fit(x_train,
                    y_train,
                    epochs=50,
                    batch_size=64,
                    validation_data=(x_val, y_val),
                    callbacks=callbacks)

In [ ]:
history_dict = history3.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, 'bo', label='Потери при Обучении')
plt.plot(epochs, val_loss_values, 'b', label='Потери при Проверке')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.clf()
acc = history_dict["accuracy"]
val_acc = history_dict["val_accuracy"]
plt.plot(epochs, acc, "bo", label="Точность на этапе обучения")
plt.plot(epochs, val_acc, "b", label="Точность на этапе проверки")
plt.title("Точность")
plt.xlabel("Эпохи")
plt.ylabel("Точность")
plt.legend()
plt.show()

plt.clf()
acc = history_dict["precision"]
val_acc = history_dict["val_precision"]
plt.plot(epochs, acc, "bo", label="Точность на этапе обучения")
plt.plot(epochs, val_acc, "b", label="Точность на этапе проверки")
plt.title("Точность")
plt.xlabel("Эпохи")
plt.ylabel("Точность")
plt.legend()
plt.show()

plt.clf()
acc = history_dict["recall"]
val_acc = history_dict["val_recall"]
plt.plot(epochs, acc, "bo", label="Точность на этапе обучения")
plt.plot(epochs, val_acc, "b", label="Точность на этапе проверки")
plt.title("Точность")
plt.xlabel("Эпохи")
plt.ylabel("Точность")
plt.legend()
plt.show()

## RuBERT


In [ ]:
from transformers import pipeline
model = pipeline(model="seara/rubert-tiny2-ru-go-emotions")

## LaBSE